<a href="https://colab.research.google.com/github/marjankamyab/ezafe_data_analysis/blob/main/Ezafe_Twitter_Data_Prepration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Mar 19 11:50:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Get Started with LSCP
LSCP: Enhanced Large Scale Colloquial Persian Language Understanding <br>

<img src="https://www.researchgate.net/publication/339945878/figure/fig1/AS:869541483663360@1584326253493/A-sample-of-the-extracted-sentence-with-dependency-relations-in-syntactic-annotation.png" alt="A sample of the extracted sentence with dependency relations in syntactic annotation, part-of-speech tags, sentiment polarity and translations." align="right" style="width:15%; padding:4em;">

Learn more about this study at https://iasbs.ac.ir/~ansari/lscp/ <br>

> LSCP is hierarchically organized in asemantic taxonomy that focuses on multi-task informal Persian language understanding as a comprehensive problem. This encompasses the recognition of multiple semantic aspects in the human-level sentences, which naturally captures from the real-world sentences. The LSCP has 120M sentences from 27M casual Persian tweets with its derivation tree, part-of-speech tags, sentiment polarity and translations in English, German, Czech, Italian and Hindi spoken languages.

### Bibtex

<pre style="font-size:12px;">@InProceedings{abdikhojasteh:2020:LREC,<br>  author    = {Abdi Khojasteh, Hadi  and  Ansari, Ebrahim  and  Bohlouli, Mahdi},<br>  title     = {LSCP: Enhanced Large Scale Colloquial Persian Language Understanding},<br>  booktitle = {Proceedings of The 12th Language Resources and Evaluation Conference},<br>  month     = {May},<br>  year      = {2020},<br>  address   = {Marseille, France},<br>  publisher = {European Language Resources Association},<br>  pages     = {6323--6327},<br>  url       = {https://www.aclweb.org/anthology/2020.lrec-1.776}<br>}</pre>

<pre style="font-size:12px;">@misc{11234/1-3195,<br>  title  = {Large-Scale Colloquial Persian 0.5},<br>  author = {Abdi Khojasteh, Hadi and Ansari, Ebrahim and Bohlouli, Mahdi},<br>  url    = {http://hdl.handle.net/11234/1-3195},<br>  note   = {{LINDAT}/{CLARIAH}-{CZ} digital library at the Institute of Formal and Applied Linguistics ({{\'U}FAL}), Faculty of Mathematics and Physics, Charles University},<br>  copyright = {Creative Commons - Attribution-{NonCommercial}-{NoDerivatives} 4.0 International ({CC} {BY}-{NC}-{ND} 4.0)},<br>  year   = {2020} }</pre>



In [2]:
from google.colab import drive
drive_dir = '/content/drive/My Drive'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import pandas as pd
import regex as re
from collections import defaultdict

In [130]:
output_dir = os.path.join(drive_dir, 'Persian', 'persian_twitter_adjv_1M.csv')
overlap_dir = os.path.join(drive_dir, 'Persian', 'persian_twitter_overlap_1M.csv')

## Download and prepare dataset

In [5]:
#Download full corpus from LINDAT repository
!curl --remote-name-all https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3195{/lscp-0.5-fa-derivation-tree.7z}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  502M  100  502M    0     0  10.1M      0  0:00:49  0:00:49 --:--:-- 11.1M


In [6]:
#Decompress all files
!7z -y e "lscp-0.5-fa-derivation-tree.7z"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 527048817 bytes (503 MiB)

Extracting archive: lscp-0.5-fa-derivation-tree.7z
--
Path = lscp-0.5-fa-derivation-tree.7z
Type = 7z
Physical Size = 527048817
Headers Size = 170
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      0% - lscp-0.5-fa-derivation-tree.txt                                        1% - lscp-0.5-fa-derivation-tree.txt                                        2% - lscp-0.5-fa-derivation-tree.txt                                        3% - lscp-0.5-fa-derivation-tree.txt

In [7]:
#Open the file with read only permit
f = open('lscp-0.5-fa-derivation-tree.txt', "r")
#Use readlines to read all lines in the file
#The variable "lines" is a list containing all lines in the file
lines = f.readlines()
#Close the file after reading the lines.
f.close()

In [8]:
#Select an example
example_text = lines[0]
print(example_text)

[('به', 'P'), ('نظر', 'Ne'), ('شما', 'PRO'), ('وقتی', 'CONJ'), ('وزیر', 'Ne'), ('امورخارجه', 'Ne'), ('بریتانیا', 'N'), ('،', 'PUNC'), ('برای', 'Pe'), ('آزادی', 'N'), ('اینقدر', 'ADV'), ('تقلا', 'N'), ('و', 'CONJ'), ('پروپاگاندا', 'N'), ('می\u200cکند', 'V'), ('،', 'PUNC'), ('در', 'P'), ('حال', 'Ne'), ('ارسا…', 'N')]



##Text preprocessing

In [9]:
sents = []
for line in lines[:1000000]:
  line = line.replace('ي', 'ی').replace('ك', 'ک').replace('ة', 'ه').replace('\\', '')
  line = re.sub(r'u200[cd]', '', line)
  words = line.split(', (')
  sent = []
  for word in words:          
    if word.startswith('['):
      word = word[3:]

    else:
      word = word[1:]
                
    if not word.endswith(']\n'):
      word = word[:-2]

    else:
      word = word[:-3]

    sent += [word]
  sents += [sent]

In [10]:
delim = "', '"
for i, sent in enumerate(sents):
  for j, word in enumerate(sent):  
    if delim in word:
      sents[i][j] = word.split(delim)

In [11]:
for i,sent in enumerate(sents):
  for j,s in enumerate(sent):
    if len(s) != 2:
      s = s.replace('\\', '')
      sents[i][j] = s.split(', ')

In [12]:
sents[0][0]

['به', 'P']

In [44]:
tags, lst = [], []
for sent in sents:
  text = " ".join([word[0] for word in sent if len(word)==2])
  for word in sent:
    if len(word) == 2:
      tag = word[1].replace("\'", '')
      tags.append(tag)
      if tag in ['AJ', 'AJe', 'ADV']:
        if tag == 'AJe':
          tag = 'AJ'
        lst.append([word[0], tag, text, 0, 0])

In [91]:
lst[0]

['نه',
 'ADV',
 'آره خیلی شباهت داره با اقتدار کلیسا ولی از اونجایی فواره چون بلند شود سرنگون گردد ظلم هم بعد اوج گرفتن شروع به تموم شدن میکنه ؛ بزرگترین بدی انقلابها اینکه میشن صحنه انتقام و عقده گشایی نه اصلاح کارها ولی خب فعلا همه دنبالشن چون سریع اتفاق میوفته',
 0,
 39740]

In [92]:
len(lst)

1942552

In [97]:
lst = [tup for tup in set(tuple(i) for i in lst)]

In [117]:
total_words = len(lst)

In [99]:
adj_count, adv_count = {}, {}

for word, tag, txt, ct1, ct2 in lst:
  if tag == 'AJ':
    if word not in adj_count:
      adj_count[word] = 1
    else:
      adj_count[word] += 1

  if tag == 'ADV':
    if word not in adv_count:
      adv_count[word] = 1
    else:
      adv_count[word] += 1

In [46]:
for i, (word, tag, txt, ct1, ct2) in enumerate(lst):
  if word in adj_count:
    lst[i][3] = adj_count[word]

  if word in adv_count:
    lst[i][4] = adv_count[word]


In [100]:
lst[3]

('آخه', 'ADV', 'وصله یه این زیبایی آخه', 0, 4212)

In [101]:
lst = sorted(lst, key=lambda x: (x[4], x[3]), reverse=True)
lst[0]

('نه',
 'ADV',
 'آقای خمینی گفت ؛ نباید کسانی در گذشته برای حال فعلی مردم ، تصمیم بگیرند ! سال پیش برای حال فعلی تصمیم گرفته_شد ! اینک یک سوال ساده و غیر مسلحانه ! آقای خامنهایی شما نه مرجع تقلید من هستید ، نه سیاستهای شما را بعنوان میپسندم ، تکلیف من و امثال من چیست ؟',
 0,
 39740)

In [115]:
tags = sorted(set(tags))
tags

['ADV',
 'ADVe',
 'AJ',
 'AJe',
 'CL',
 'CONJ',
 'CONJe',
 'DET',
 'DETe',
 'INT',
 'N',
 'NUM',
 'NUMe',
 'Ne',
 'P',
 'POSTP',
 'PRO',
 'PROe',
 'PUNC',
 'Pe',
 'RES',
 'RESe',
 'V']

In [104]:
len(adj_count)

45647

In [105]:
len(adv_count)

11265

In [106]:
df = pd.DataFrame(lst, columns=['word', 'tag', 'text', 'adj_count', 'adv_count'])


In [107]:
df['adj_count'] = df['adj_count'].astype('int32')
df['adv_count'] = df['adv_count'].astype('int32')

In [108]:
len(df)

963833

In [109]:
df['tag'].unique()

array(['ADV', 'AJ'], dtype=object)

In [110]:
df

,word,tag,text,adj_count,adv_count
0,نه,ADV,آقای خمینی گفت ؛ نباید کسانی در گذشته برای حال...,0,39740
1,نه,ADV,نه خب اونم راست میگفت ، منم زیاد چوسناله میکنم...,0,39740
2,نه,ADV,آدم ازدیونه انتظاری نداره به همین جهت دائم سوپ...,0,39740
3,نه,ADV,زیاد میشنویم ، نه ؟,0,39740
4,نه,ADV,دارن برای عید ، مسافرت دسته جمعی برنامه ریزی م...,0,39740
...,...,...,...,...,...
963828,سینک,AJ,صبح زود برسید و ببینید خدماتی تو ماگتون شیر تی...,1,0
963829,نبرد👥یارشان,AJ,چفیهای همراز مردان نبرد👥یارشان بودی تو در شبها...,1,0
963830,اردشیر😂😂😂از,AJ,به جان اردشیر😂😂😂از نیناش بپرس😂😂😂😂,1,0
963831,جنبان,AJ,من نمیدونستم که به خمینى سلسله جنبان هم میگویند,1,0


In [125]:
df = df[~df['adj_count'].isin(range(1,5))]
df = df[~df['adv_count'].isin(range(1,5))]

In [126]:
df = df.drop_duplicates()

In [127]:
df['total_adj'] = len(df[df['tag'] == 'AJ'])
df['total_adv'] = len(df[df['tag'] == 'ADV'])
df['total_words'] = total_words
df['total_adj'] = df['total_adj'].astype('int32')
df['total_adv'] = df['total_adv'].astype('int32')
df['total_words'] = df['total_words'].astype('int32')

In [128]:
df

,word,tag,text,adj_count,adv_count,total_adj,total_adv,total_words
0,نه,ADV,آقای خمینی گفت ؛ نباید کسانی در گذشته برای حال...,0,39740,459572,296707,963833
1,نه,ADV,نه خب اونم راست میگفت ، منم زیاد چوسناله میکنم...,0,39740,459572,296707,963833
2,نه,ADV,آدم ازدیونه انتظاری نداره به همین جهت دائم سوپ...,0,39740,459572,296707,963833
3,نه,ADV,زیاد میشنویم ، نه ؟,0,39740,459572,296707,963833
4,نه,ADV,دارن برای عید ، مسافرت دسته جمعی برنامه ریزی م...,0,39740,459572,296707,963833
...,...,...,...,...,...,...,...,...
923257,اجرائی,AJ,جالب اینجاست که « ستاد اجرائی فرمان امام » فقط...,5,0,459572,296707,963833
923258,استحتی,AJ,کار غلط ، همیشه غلط است ! حتی اگر همه همان کار...,5,0,459572,296707,963833
923259,بذاری,AJ,شما فارسهای فاشیست چقدر پررو تشریف دارین معلوم...,5,0,459572,296707,963833
923260,تیرخلاصی,AJ,درخواست سلب تابعیت آقای طاهری که اولین تبعه ای...,5,0,459572,296707,963833


In [129]:
overlap = df[(df['adj_count'] > 0) & (df['adv_count'] > 0)]
overlap

,word,tag,text,adj_count,adv_count,total_adj,total_adv,total_words
68270,دیگه,ADV,انتظار داشتی بازی باشه🙄بچه بودی دیگه پیش میاد🤗,94,25010,459572,296707,963833
68271,دیگه,ADV,جو میگرشونیه مدت که میگذره دنبال رفقای فراموش ...,94,25010,459572,296707,963833
68272,دیگه,ADV,چی میگی شما کاندوم نمیفروشن جمعیت زیاد شه به ا...,94,25010,459572,296707,963833
68273,دیگه,ADV,وا بگو دیگه مونیک جیکار کردم😢😢👀,94,25010,459572,296707,963833
68274,دیگه,ADV,دوست ندارن دیگه : /😢,94,25010,459572,296707,963833
...,...,...,...,...,...,...,...,...
512811,چیزى,AJ,اکثریت عظیم روشنفکران تحصیل کردهای که مىشناسم ...,5,5,459572,296707,963833
512812,سالشونه,AJ,من نمیدونم چرا دوستای ایم سعی دارن حتما بگن سا...,5,5,459572,296707,963833
512813,مامانه,ADV,هستش واقعی جن دیدی 👻👻👻👻👻👻از اوناییکه خون گردن ...,5,5,459572,296707,963833
512814,دونفری,AJ,تخت یه نفره اییم که روش دونفری بخوابی بازم شکن...,5,5,459572,296707,963833


In [134]:
sorted(list(overlap['word'].unique()))

['',
 'u2067',
 'u2069',
 'u2069u2067',
 '؟؟',
 '؟؟؟',
 'آتیش',
 'آخوندا',
 'آخوندی',
 'آدما',
 'آرام',
 'آره',
 'آرومتر',
 'آزاد',
 'آزادانه',
 'آزمایشی',
 'آسان',
 'آسوده',
 'آسون',
 'آسونه',
 'آشپزخونه',
 'آشکارتر',
 'آقایون',
 'آموز',
 'آهسته',
 'آواتار',
 'آگاهانه',
 'ابلهانه',
 'اتفاقی',
 'اجباری',
 'احمقانه',
 'اخر',
 'اخه',
 'ارزونتر',
 'اره',
 'از',
 'ازشون',
 'ازین',
 'اشتباه',
 'اشتباهی',
 'اشرافی',
 'اصن',
 'الله',
 'الکی',
 'امنه',
 'انداز',
 'اندکی',
 'انسانه',
 'انقد',
 'انقدر',
 'انچه',
 'انگاری',
 'اوردوز',
 'اون',
 'اونجا',
 'اونقدر',
 'اونموقع',
 'اونه',
 'اونورتر',
 'اى',
 'اگه',
 'ایرانه',
 'ایستاده',
 'اینبار',
 'اینتر',
 'اینجا',
 'اینجور',
 'اینجوری',
 'اینطوری',
 'اینقد',
 'اینقده',
 'اینه',
 'اینور',
 'اینکار',
 'ایول',
 'باباش',
 'باحاله',
 'بارسا',
 'باز',
 'بازم',
 'بازه',
 'بازیچه',
 'بالا',
 'بالاتر',
 'بالقوه',
 'بامزه',
 'باهاش',
 'باهاشون',
 'باهم',
 'ببره',
 'ببینه',
 'بتونه',
 'بتونی',
 'بجا',
 'بحق',
 'بخدا',
 'بخره',
 'بخون',
 'بخونه',
 'بد',
 'بدت

In [131]:
df.to_csv(output_dir)
overlap.to_csv(overlap_dir)